## Classification of Anomalies  in Gastrointestinal Tract through Endoscopic Imagery with Deep Learning

### Transfer learning using InceptionV3 model

In [27]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Dense
from keras.utils import get_file
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

#### Generate data from local kvasir-dataset
kvasir-dataset folder should be in the same folder as the Inception_V3_Model

In [8]:
path ='kvasir-dataset' 

image_gen = ImageDataGenerator(
    rescale=1/255,fill_mode='nearest', 
    brightness_range= [0.4,1],
    validation_split = 0.2,
    zoom_range=0.2,)

train_generator = image_gen.flow_from_directory(
    path,
    target_size=(224,224),
    batch_size=32,
    subset="training",
    seed=1337,
    class_mode='categorical'
)

validation_generator = image_gen.flow_from_directory(
    path,
    target_size=(224,224),
    batch_size=32,
    subset="validation",
    seed=1337,
    class_mode='categorical'
)

Found 3200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [24]:
vgg19_model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

for layer in vgg19_model.layers[:30]:
        layer.trainable = True
vgg19_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [28]:
x = vgg19_model.output
x =  tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
predictions = tf.keras.layers.Dense(8, activation="softmax")(x)   
model = tf.keras.models.Model(vgg19_model.input,predictions)

model.compile(optimizer =  optimizers.SGD(lr=0.0001, momentum=0.9),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

#### Training model

In [ ]:
epochs = 50

history = model.fit(
            train_generator,
            batch_size = 8,
            validation_data = validation_generator,
            epochs = epochs,
            verbose = 1)

Epoch 1/50
 35/100 [=========>....................] - ETA: 1:14:13 - loss: 1.4518 - accuracy: 0.4161

#### Print training and validation accuracy

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()